<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Answer Relevancy and Context Relevancy Evaluations</h1>
<h5 align="center">👉 See Llama-Index <a href="https://github.com/run-llama/llama_index/blob/80cee5a511360eedd7837f20d283bf0a9bd05603/docs/docs/examples/evaluation/answer_and_context_relevancy.ipynb">notebook</a> for more info 👈</h5>


<a href="https://colab.research.google.com/github/arize-ai/phoenix/blob/main/tutorials/experiments/llama-index/answer_and_context_relevancy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
%pip install -Uqqq "arize-phoenix[llama-index]>=4.6" nest_asyncio

# Enter OpenAI API Key

In [ ]:
import os
from getpass import getpass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")

# Import Modules

In [ ]:
import json
import tempfile
from textwrap import shorten
from time import time_ns
from typing import Tuple

import nest_asyncio
from llama_index.core import VectorStoreIndex
from llama_index.core.evaluation import AnswerRelevancyEvaluator, ContextRelevancyEvaluator
from llama_index.core.llama_dataset import download_llama_dataset
from llama_index.llms.openai import OpenAI
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

import phoenix as px
from phoenix.experiments import evaluate_experiment, run_experiment
from phoenix.experiments.types import Explanation, Score
from phoenix.otel import register

nest_asyncio.apply()

# Launch Phoenix

In [ ]:
px.launch_app().view()

# Instrument Llama-Index

In [ ]:
tracer_provider = register(endpoint="http://127.0.0.1:4317")
LlamaIndexInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

# Upload Dataset to Phoenix

In [ ]:
sample_size = 7
dataset_name = "EvaluatingLlmSurveyPaperDataset"
with tempfile.TemporaryDirectory() as dir_name:
    rag_dataset, documents = download_llama_dataset(dataset_name, dir_name)
dataset = px.Client().upload_dataset(
    dataset_name=f"{dataset_name}_{time_ns()}",
    dataframe=rag_dataset.to_pandas().sample(sample_size, random_state=42),
)

# Dataset Can be Viewed as Dataframe

In [ ]:
dataset.as_dataframe()

# Take a Look at the Data Structure of an Example

In [ ]:
dataset[0]

# Define Task Function on Examples

Task function can be either sync or async.

In [ ]:
index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()


async def task(input):
    ans = await query_engine.aquery(input["query"])
    return {
        "contexts": [node.text for node in ans.source_nodes],
        "response": ans.response,
    }

# Check that Task Can Run Successfully

In [ ]:
example = dataset[0]
task_output = await task(example.input)
print(shorten(json.dumps(task_output), width=80))

# Dry-Run Experiment

On 3 randomly selected examples

In [ ]:
experiment = run_experiment(dataset, task, dry_run=3)

# Experiment Results Can be Viewed as Dataframe

In [ ]:
experiment.as_dataframe()

# Take a Look at the Data Structure of an Experiment Run

In [ ]:
experiment[0]

# Define Evaluators For Each Experiment Run

Evaluators can be sync or async.

Function arguments `output` and `input` refers to the attributes of the same name in the `ExperimentRun` data structure shown above.

In [ ]:
async def answer_relevancy(output, input) -> Tuple[Score, Explanation]:
    ans = await AnswerRelevancyEvaluator(
        llm=OpenAI(temperature=0, model="gpt-4o"),
    ).aevaluate(input["query"], response=output["response"])
    return ans.score, ans.feedback


async def context_relevancy(output, input) -> Tuple[Score, Explanation]:
    ans = await ContextRelevancyEvaluator(
        llm=OpenAI(temperature=0, model="gpt-4o"),
    ).aevaluate(input["query"], contexts=output["contexts"])
    return ans.score, ans.feedback


evaluators = [answer_relevancy, context_relevancy]

# Check that Evals Can Run Successfully

In [ ]:
run = experiment[0]
example = dataset.examples[run.dataset_example_id]
for fn in (answer_relevancy, context_relevancy):
    _ = await fn(run.output, example.input)
    print(fn.__qualname__)
    print(shorten(json.dumps(_), width=80))

# Run Evaluations

In [ ]:
experiment = evaluate_experiment(experiment, evaluators)

# Evaluation Results Can be Viewed as Dataframe

In [ ]:
experiment.get_evaluations()

# Run Task and Evals Together

In [ ]:
_ = run_experiment(dataset, task, evaluators)